- Question

  - 영화리뷰 긍/부정 예측 Estimator pipeline을 테스트 (trainRatio=0.8)
  - Word2Vec의 파라미터 vectorSize를 5, 10, 20, 40으로 바꾸며 정확도를 측정하여 출력
  - 매뉴얼 및 웹검색을 통해 문제해결!

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.appName('my app').master('local[8]').getOrCreate()
sc = spark.sparkContext

22/10/27 17:09:13 WARN Utils: Your hostname, bagdoyeong-ui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.0.13 instead (on interface en0)
22/10/27 17:09:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/27 17:09:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/27 17:09:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/27 17:09:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, Word2Vec, VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param

In [32]:
schema = StructType([
    StructField('text', StringType(), True),
    StructField('label', FloatType(), True)
])

data = spark.read.format('csv').option('escape', "\"").option('header', 'true').schema(schema).load('input_week09/imdb-review-sentiment.csv')
data.printSchema()

root
 |-- text: string (nullable = true)
 |-- label: float (nullable = true)



In [33]:
data.select('text', 'label').show()

+--------------------+-----+
|                text|label|
+--------------------+-----+
|I grew up (b. 196...|  0.0|
|When I put this m...|  0.0|
|Why do people who...|  0.0|
|Even though I hav...|  0.0|
|Im a die hard Dad...|  1.0|
|A terrible movie ...|  0.0|
|Finally watched t...|  1.0|
|I caught this fil...|  0.0|
|It may be the rem...|  1.0|
|My Super Ex Girlf...|  1.0|
|I can't believe p...|  1.0|
|If you haven't se...|  0.0|
|I have always bee...|  1.0|
|Greg Davis and Br...|  0.0|
|A half-hearted at...|  0.0|
|If you want a fun...|  1.0|
|I really wanted t...|  1.0|
|The main problem ...|  0.0|
|The folks at Disn...|  0.0|
|A friend told me ...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [20]:
stopwords = ['i', 'the', 'she', 'he', 'have', 'not']

# df = spark.createDataFrame([
#     ('This movie was so poorly written and directed I fell asleep 30minutes through the movie.', 0),
#     ('The most interesting thing about Miryang (Secret Sunshine) is the actores.', 1),
#     ('William Hurt may not be an American matinee idol anymorem but he still has pretty good talent.', 1)
# ], ['text', 'sentiment'])

In [21]:
class RemoveStopWordsAndSpecialCharacters(Transformer, HasInputCol, HasOutputCol):
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, stopwords=None):
        super(RemoveStopWordsAndSpecialCharacters, self).__init__()
        self.stopwords = Param(self, 'stopwords', '')
        self._setDefault(stopwords=set())
        kwargs = self._input_kwargs
        self._set(**kwargs)
        
    def setStopwords(self, value):
        self._paramMap[self.stopwords] = value
        return self
    
    def getStopwords(self):
        return self.getOrDefault(self.stopwords)
    
    def _transform(self, dataset):
        stopwords = self.getStopwords()
        
        def f(s):
            return [ ''.join(e for e in token if e.isalnum()) for token in s if token not in stopwords]
        
        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [93]:
tokenizer = Tokenizer(inputCol='text', outputCol='words')
cleaning = RemoveStopWordsAndSpecialCharacters(inputCol='words', outputCol='clean_words', stopwords=stopwords)
# hashingTF = HashingTF(inputCol='clean_words', outputCol='tf')
w2v = Word2Vec(vectorSize=10, inputCol='clean_words', outputCol='w2v', minCount=1, maxIter=1, )
asm = VectorAssembler(inputCols = [w2v.getOutputCol()], outputCol='features')
svm = LinearSVC(labelCol='label', maxIter=5)

mypipeline = Pipeline(stages=[tokenizer, cleaning, w2v, asm, svm])
df10 = mypipeline.fit(data).transform(data)

In [100]:
tokenizer = Tokenizer(inputCol='text', outputCol='words')
cleaning = RemoveStopWordsAndSpecialCharacters(inputCol='words', outputCol='clean_words', stopwords=stopwords)
# hashingTF = HashingTF(inputCol='clean_words', outputCol='tf')
w2v = Word2Vec(vectorSize=15, inputCol='clean_words', outputCol='w2v', minCount=1, maxIter=1, )
asm = VectorAssembler(inputCols = [w2v.getOutputCol()], outputCol='features')
svm = LinearSVC(labelCol='label', maxIter=5)

mypipeline = Pipeline(stages=[tokenizer, cleaning, w2v, asm, svm])
df15 = mypipeline.fit(data).transform(data)

In [101]:
tokenizer = Tokenizer(inputCol='text', outputCol='words')
cleaning = RemoveStopWordsAndSpecialCharacters(inputCol='words', outputCol='clean_words', stopwords=stopwords)
# hashingTF = HashingTF(inputCol='clean_words', outputCol='tf')
w2v = Word2Vec(vectorSize=20, inputCol='clean_words', outputCol='w2v', minCount=1, maxIter=1, )
asm = VectorAssembler(inputCols = [w2v.getOutputCol()], outputCol='features')
svm = LinearSVC(labelCol='label', maxIter=5)

mypipeline = Pipeline(stages=[tokenizer, cleaning, w2v, asm, svm])
df20 = mypipeline.fit(data).transform(data)

In [98]:
# size = 5

df_result = df.withColumn('prediction', df['prediction'].cast(FloatType()))
df_result.selectExpr('label == prediction').filter('(label = prediction) == True').count() / df_result.count() * 100

61.834999999999994

In [99]:
# size = 10

df_result = df10.withColumn('prediction', df10['prediction'].cast(FloatType()))
df_result.selectExpr('label == prediction').filter('(label = prediction) == True').count() / df_result.count() * 100

65.22

In [102]:
# size = 15

df_result = df15.withColumn('prediction', df15['prediction'].cast(FloatType()))
df_result.selectExpr('label == prediction').filter('(label = prediction) == True').count() / df_result.count() * 100

69.0375

In [103]:
# size = 20

df_result = df20.withColumn('prediction', df20['prediction'].cast(FloatType()))
df_result.selectExpr('label == prediction').filter('(label = prediction) == True').count() / df_result.count() * 100

72.1